In [17]:
#Import libraries
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib.ticker import PercentFormatter
from operator import attrgetter
import matplotlib.colors as mcolors
warnings.filterwarnings("ignore")
import matplotlib.ticker as tkr

#Use fivethirtyeight style
plt.style.use('fivethirtyeight')

#Cancel any style and reset matplotlib styles
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

In [18]:
fname = 'C:\\tmp\\workdir\\BBdata\\GitHub\\Climate.Change.Project2\\Data\\Dataset_Climate.xlsx'
df = pd.read_excel(fname)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LineNo            8950 non-null   int64 
 1   DisNo             8950 non-null   object
 2   Year              8950 non-null   int64 
 3   DisasterSubgroup  8950 non-null   object
 4   DisasterType      8950 non-null   object
 5   DisasterSubtype   7655 non-null   object
 6   Country           8950 non-null   object
 7   ISO               8950 non-null   object
 8   Region            8950 non-null   object
 9   Continent         8950 non-null   object
 10  TotalDamages      8950 non-null   int64 
dtypes: int64(3), object(8)
memory usage: 769.3+ KB


,LineNo,DisNo,Year,DisasterSubgroup,DisasterType,DisasterSubtype,Country,ISO,Region,Continent,TotalDamages
0,1,2000-0372-CHN,2000,Hydrological,Flood,Riverine flood,China,CHN,Eastern Asia,Asia,19
1,2,2000-9186-AFG,2000,Climatological,Drought,Drought,Afghanistan,AFG,Southern Asia,Asia,326
2,3,2000-0373-BGD,2000,Hydrological,Flood,Flash flood,Bangladesh,BGD,Southern Asia,Asia,145
3,4,2000-0905-BOL,2000,Hydrological,Flood,NaN,Bolivia (Plurinational State of),BOL,South America,Americas,145
4,5,2000-0019-BRA,2000,Hydrological,Flood,Riverine flood,Brazil,BRA,South America,Americas,145


In [19]:
df['Recency'] = (max(df['Year']) - df['Year'])
df.head()

,LineNo,DisNo,Year,DisasterSubgroup,DisasterType,DisasterSubtype,Country,ISO,Region,Continent,TotalDamages,Recency
0,1,2000-0372-CHN,2000,Hydrological,Flood,Riverine flood,China,CHN,Eastern Asia,Asia,19,21
1,2,2000-9186-AFG,2000,Climatological,Drought,Drought,Afghanistan,AFG,Southern Asia,Asia,326,21
2,3,2000-0373-BGD,2000,Hydrological,Flood,Flash flood,Bangladesh,BGD,Southern Asia,Asia,145,21
3,4,2000-0905-BOL,2000,Hydrological,Flood,NaN,Bolivia (Plurinational State of),BOL,South America,Americas,145,21
4,5,2000-0019-BRA,2000,Hydrological,Flood,Riverine flood,Brazil,BRA,South America,Americas,145,21


In [132]:
df_rfm = df.groupby('Country').agg({'Recency':'min'
                                           ,'Year':pd.Series.nunique
                                           ,'TotalDamages':'sum'})
df_rfm.head(20)


,Recency,Year,TotalDamages
Country,,,
Afghanistan,0,22,33522
Albania,0,13,6301
Algeria,0,19,18834
American Samoa,12,4,1727
Angola,0,21,7560
Anguilla,4,1,200
Antigua and Barbuda,4,3,839
Argentina,0,22,23145
Armenia,1,7,2146


In [83]:
df_rfm.rename(columns={'Year':'Frequency','TotalDamages':'Monetary'},inplace=True)

In [85]:
df_rfm

,Recency,Frequency,Monetary
DisasterType,,,
Animal accident,7,1,0
Drought,0,22,206263
Earthquake,0,22,926333
Epidemic,0,22,0
Extreme temperature,0,22,85970
Flood,0,22,940591
Glacial lake outburst,0,1,315
Impact,8,1,33
Insect infestation,1,8,232


In [86]:
df_rfm["recency_score"] = pd.qcut(df_rfm['Recency'], q = 12, labels=[4, 3, 2, 1], duplicates = 'drop')
df_rfm["recency_score"] = df_rfm.recency_score.astype(int)

In [91]:
df_rfm["frequency_score"] = pd.qcut(df_rfm['Frequency'], q = 10, labels=[4, 3, 2, 1], duplicates = 'drop')
df_rfm["frequency_score"] = df_rfm.frequency_score.astype(int)


In [93]:
df_rfm["monetary_score"] = pd.qcut(df_rfm['Monetary'], q = 4, labels=[4, 3, 2, 1], duplicates = 'drop')
df_rfm["monetary_score"] = df_rfm.monetary_score.astype(int)

In [96]:
df_rfm["RFM_Segment"] = df_rfm.recency_score * 100 + df_rfm.frequency_score * 10 + df_rfm.monetary_score

In [97]:
df_rfm["RFM_Score"] = df_rfm.recency_score + df_rfm.frequency_score  + df_rfm.monetary_score

In [98]:
df_rfm

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score
DisasterType,,,,,,,,
Animal accident,7,1,0,1,4,4,144,9
Drought,0,22,206263,4,1,1,411,6
Earthquake,0,22,926333,4,1,1,411,6
Epidemic,0,22,0,4,1,4,414,9
Extreme temperature,0,22,85970,4,1,2,412,7
Flood,0,22,940591,4,1,1,411,6
Glacial lake outburst,0,1,315,4,4,3,443,11
Impact,8,1,33,1,4,4,144,9
Insect infestation,1,8,232,3,3,3,333,9


In [130]:
segt_map = {
    r'1[1-2-3-4]1': 'VIP',
    r'[2-3-4][1-2-3-4]1': 'Top Recent',
    r'1[1-2-3-4]1': 'Top at Risk ',

    
    
    r'[3-4][3-4]3': 'High Promising',
    r'[2-3-4][1-2]3': 'High New',
    r'2[3-4]3': 'High Loyal',

    
    
    r'[3-4][3-4]2': 'Medium Potential',
    r'[2-3-4][1-2]2': 'Medium New',
    r'2[3-4]2': 'Medium Loyal',

    
    
    r'4[1-2-3-4]1': 'Low New',
    r'[2-3][1-2-3-4]1': 'Low Loyal',
    
    r'1[1-2-3-4][1-2-3]': 'Need Activation'
}
df_rfm['Segment_labels'] = df_rfm['RFM_Segment']
df_rfm['Segment_labels'] = df_rfm['Segment_labels'].astype(str).replace(segt_map, regex=True)



In [131]:
df_rfm

,level_0,index,DisasterType,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score,Segment_labels
0,0,0,Animal accident,7,1,0,1,4,4,144,9,144
1,1,1,Drought,0,22,206263,4,1,1,411,6,Top Recent
2,2,2,Earthquake,0,22,926333,4,1,1,411,6,Top Recent
3,3,3,Epidemic,0,22,0,4,1,4,414,9,414
4,4,4,Extreme temperature,0,22,85970,4,1,2,412,7,Medium New
5,5,5,Flood,0,22,940591,4,1,1,411,6,Top Recent
6,6,6,Glacial lake outburst,0,1,315,4,4,3,443,11,High Promising
7,7,7,Impact,8,1,33,1,4,4,144,9,144
8,8,8,Insect infestation,1,8,232,3,3,3,333,9,High Promising
9,9,9,Landslide,0,22,8734,4,1,2,412,7,Medium New


In [127]:
segt_map

{'144': 'VIP',
 '[2-3-4][1-2-3-4]1': 'Top Recent',
 '1[1-2-3-4]1': 'Top at Risk ',
 '[3-4][3-4]3': 'High Promising',
 '[2-3-4][1-2]3': 'High New',
 '2[3-4]3': 'High Loyal',
 '[3-4][3-4]2': 'Medium Potential',
 '[2-3-4][1-2]2': 'Medium New',
 '2[3-4]2': 'Medium Loyal',
 '4[1-2-3-4]1': 'Low New',
 '[2-3][1-2-3-4]1': 'Low Loyal',
 '1[1-2-3-4][1-2-3]': 'Need Activation'}